In [1]:
import json, numbers, math, time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from elfpy.simulators import YieldSimulator
from elfpy.utils.data import format_trades

In [2]:
pd.set_option("float_format",'{:,.6f}'.format)

random_seed = 3
config = {
    'min_fee': 0.,
    'max_fee': 0.5,
    'floor_fee': 0,
    't_min': 0.001,
    't_max': 1.0,
    'base_asset_price': 2500., # aka market price
    'min_target_liquidity': 100000.,
    'max_target_liquidity': 10000000.,
    'min_target_volume': 2e5,
    'max_target_volume': 6e5,
    'min_pool_apy': 0.5,
    'max_pool_apy': 50,
    'min_vault_age': 0.,
    'max_vault_age': 2,
    'min_vault_apy': 0.,
    'max_vault_apy': 0.1,
    'precision': 8,
    'pricing_model_name': 'YieldSpace',
    'tokens': ['base', 'fyt'],
    'trade_direction': 'out',
    'pool_duration': 180,
    'num_trading_days': 180, # should be <= days_until_maturity
    'rng': np.random.default_rng(random_seed),
    'verbose': True,
}

In [3]:
start_time = time.time()
num_runs = 0

simulator = YieldSimulator(**config)
simulator.set_random_variables()

apr = [0.10,]*config['num_trading_days']
#apr[config['num_trading_days']//2:(config['num_trading_days']//2) + 5] == [0.08, 0.08, 0.08]
#apr[config['num_trading_days']//2:] = [0.08,]*(config['num_trading_days']//2)
#apr[config['num_trading_days']//2:] = [0.01,]*(config['num_trading_days']//2)

override_dict = {
    'pricing_model_name': 'Element',
    'vault_apy': apr,
    'init_pool_apy' : apr[0],
    'init_vault_age': 1,
    'target_daily_volume': 5*1e5,
    'target_liquidity': 10*1e6,
    'fee_percent' : 0.1,
}
simulator.reset_rng(np.random.default_rng(random_seed)) # do this to make sure simulations run over the same trade sequence
simulator.run_simulation(override_dict)
num_runs += 1

floor_fee_list = [0, 5]
override_dict['pricing_model_name'] = 'YieldSpacev2'
for floor_fee in floor_fee_list:
    override_dict['floor_fee'] = floor_fee
    simulator.reset_rng(np.random.default_rng(random_seed)) # do this to make sure simulations run over the same trade sequence
    simulator.run_simulation(override_dict)
    num_runs += 1

end_time = time.time()
print(f'Total time for {num_runs} runs was {end_time-start_time:.3f} seconds; which is {(end_time-start_time)/num_runs:.3f} seconds per run')

calc_base_asset_reserves:
base_asset_reserves: 3.93607835978178e-44
calc_liquidity: 
base_asset_reserves=7.87215671956356e-44, token_asset_reserves=4197.260273972603, scaling_factor=2.0, spot_price_from_apy=0.9530026109660574, total_supply=4,000(10,000,000 USD), apy=0.10000000000000003
SIM -- INITIAL
PRICE: 0.9530026109660574; DAYS REMAINING: 180.0
trades=0 init_share_price=1.1, share_price=1.1003013698630137, amount=7.87215671956356e-44, reserves=(7.87215671956356e-44, 4197.260273972603)
total orders are less than 10.
amount=7.87215671956356e-44, token_asset+total_supply=8394.520547945205, base_asset/share_price=7.154545959116305e-44, token_in=fyt, fee_percent=0.1, time_remaining=0.00044454267678766777, init_share_price=1.1, share_price=1.1003013698630137
without_fee_or_slippage=7.502185907678067e-44, output_with_fee=7.87215671956356e-44, output_without_fee=7.87215671956356e-44, fee=0.0
SIM -- RUN: 1
PRICE: 0.0; DAYS REMAINING: 180.0


/Users/dylan/code/element/analysis/src/elfpy/pricing_models.py:287: RuntimeWarning: divide by zero encountered in double_scalars
  share_price


AssertionError: ERROR: calc_apy_from_spot_price: Price argument should be greater than zero, not 0.0

In [ ]:
[trades, trades_agg] = format_trades(simulator.analysis_dict)

In [ ]:
display(trades.groupby(['model_name', 'token_in']).agg({
    'trade_volume_usd': ['sum'],
    'run_trade_number': ['count'],
    'fee_in_bps': ['mean', 'std', 'min', 'max', 'sum'],
    'day': ['mean', 'min', 'max'],
    })
)

In [ ]:
numPlots = 2
colors = list(plt.rcParams['axes.prop_cycle'].by_key()['color'])
model_colors = {model:colors[i] for i, model in enumerate(trades_agg.model_name.unique())}

## DAY PLOTS
figsize = (24, 12)
fig, ax = plt.subplots(ncols=1, nrows=numPlots, gridspec_kw={'wspace':0, 'hspace':0, 'height_ratios':np.ones(numPlots)})
fig.patch.set_facecolor('white')   # set fig background color to white

# fees
current_plot = 0
for model in trades_agg.model_name.unique():
    model_df = trades_agg.loc[trades_agg.model_name==model, :]
    ax[current_plot] = model_df.plot(
        x='day',
        y='fee_in_usd_sum',
        figsize=figsize,
        ax=ax[current_plot],
        color=model_colors[model],
        label=model
    )
ax[current_plot].set_xlabel('')
ax[current_plot].set_ylabel('Fees (USD)', fontsize=18)
ax[current_plot].tick_params(axis='both', labelsize=18)
ax[current_plot].grid(visible=True, linestyle='--', linewidth='1', color='grey', which='both', axis='y')
ax[current_plot].xaxis.set_ticklabels([])
title = (
    f'Initial pool APY: {trades.pool_apy[0]:.2f}% '
    + f'Time Stretch: {trades.init_time_stretch[0]:.2f} '
    + f'Maturity: {trades.pool_duration[0]} days\n'
    + f'Target Liquidity: {trades.target_liquidity[0]:,.0f} '
    + f'Target Daily Volume: {trades.target_daily_volume[0]:,.0f} '
    + f'Percent Fees: {trades.fee_percent[0]:.2f}%'
)
ax[current_plot].set_title(title, fontsize=20)
ax[current_plot].legend(fontsize=18)

# fees cumulative
current_plot += 1
for model in trades_agg.model_name.unique():
    model_df = trades_agg.loc[trades_agg.model_name==model, :]
    #cumulative_fee = model_df.fee_in_usd_cum_sum.iloc[-1]
    ax[current_plot] = model_df.plot(
        x='day',
        y='fee_in_usd_cum_sum',
        figsize=figsize,
        ax=ax[current_plot],
        color=model_colors[model],
        legend=None,
        #label=f'{model}_cumulative_fee {cumulative_fee:,.0f}',
    )
ax[current_plot].set_ylabel('Cumulative Fees (USD)', fontsize=18)
ax[current_plot].tick_params(axis='both', labelsize=18)
ax[current_plot].grid(visible=True, linestyle='--', linewidth='1', color='grey', which='both', axis='y')
ax[current_plot].set_xlabel('Day', fontsize=18)

## TRADE PLOTS
num_plots = 2
figsize = (24, 12)
fig, ax = plt.subplots(ncols=1, nrows=numPlots, gridspec_kw={'wspace':0, 'hspace':0, 'height_ratios':np.ones(numPlots)})
fig.patch.set_facecolor('white')   # set fig background color to white

# pool & vault APYs
current_plot = 0
for model in trades.model_name.unique(): # for each model (per run?)
    model_df = trades.loc[trades.model_name==model, :]
    ax[current_plot] = model_df.plot(
        x='run_trade_number', # could also do num_orders if you want to know the apy for a given volume of trade
        y='pool_apy_percent',
        figsize=figsize,
        ax=ax[current_plot],
        color=model_colors[model],
        label=f'{model}'
    )
ax[current_plot] = model_df.plot( # vault is the same per model
    x='run_trade_number',
    y='vault_apy_percent',
    figsize=figsize,
    ax=ax[current_plot],
    linestyle='-',
    color='black',
    label=f'vault'
)
ax[current_plot].set_xlabel('')
ax[current_plot].set_ylabel('APY (percent)', fontsize=18)
ax[current_plot].tick_params(axis='both', labelsize=18)
ax[current_plot].grid(visible=True, linestyle='--', linewidth='1', color='grey', which='both', axis='y')
ax[current_plot].xaxis.set_ticklabels([])
ax[current_plot].legend(fontsize=18)

# price per share & spot price
current_plot += 1
for model in trades.model_name.unique():
    model_df = trades.loc[trades.model_name==model, :]
    share_pricer_growth = (
        model_df.loc[:, "share_price"].values[-1]
        / model_df.loc[:, "share_price"].values[0]
        - 1
    )
    ax[current_plot] = model_df.plot(
        x='run_trade_number',
        y='share_price',
        figsize=figsize,
        ax=ax[current_plot],
        kind='line',
        color=model_colors[model],
        label=(
            f'{model} growth={share_pricer_growth:.3%}'
        )
    )
    spot_price_growth = (
        model_df.loc[:, "pu"].values[-1]
        / model_df.loc[:, "pu"].values[0]
        - 1
    )
    ax[current_plot] = model_df.plot(
        x='run_trade_number',
        y='pu',
        figsize=figsize,
        ax=ax[current_plot],
        kind='line',
        color=model_colors[model],
        label=(
            f'{model} growth={spot_price_growth:.3%}'
        )
    )
ax[current_plot].set_ylabel('Pricing', fontsize=18)
ax[current_plot].tick_params(axis='both', labelsize=18)
ax[current_plot].grid(visible=True, linestyle='--', linewidth='1', color='grey', which='both', axis='y')
ax[current_plot].legend(fontsize=18)
ax[current_plot].set_xlabel('Trade number', fontsize=18)

plt.show()

In [ ]:
simulator.print_random_variables()